In [13]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
events = pd.read_csv('ny_events_all.csv')

In [3]:
events.head()

,Unnamed: 0,id,datetime_local,title,type,venue.id,venue.location.lat,venue.location.lon,venue.name,venue.postal_code
0,0,4421121,2018-11-08T10:00:00,Hop On Hop Off Sightseeing - New York,theater,446159,40.7050,-74.0038,Hornblower Pier 78 New York,10018
1,1,4442708,2018-11-08T11:00:00,Big Apple Circus - New York,family,3900,40.7726,-73.9841,Damrosch Park,10023
2,2,4421118,2018-11-08T12:00:00,Hop On Hop Off Sightseeing - New York,theater,446159,40.7050,-74.0038,Hornblower Pier 78 New York,10018
3,3,4575383,2018-11-08T13:00:00,Fiddler on the Roof - New York,broadway_tickets_national,57361,40.7060,-74.0186,Jewish Museum,10280
4,4,4371937,2018-11-08T14:00:00,The Phantom of the Opera - New York,broadway_tickets_national,2930,40.7584,-73.9882,Majestic Theatre,10036


In [4]:
taxi = pd.read_csv('nyc_taxi_agg.csv')

In [5]:
taxi.head()

,Unnamed: 0,PULocationID,DOLocationID,pickup_hour,dropoff_hour,pickup_dow,dropoff_dow,pickup_week,dropoff_week,sum,count
0,0,1,1,0,0,2,2,25,25,1,1
1,1,1,1,0,0,4,4,20,20,1,1
2,2,1,1,0,0,5,5,22,22,1,1
3,3,1,1,0,0,5,5,23,23,1,1
4,4,1,1,1,1,4,4,22,22,1,1


In [7]:
events['venue.postal_code'].unique()

array([10018, 10023, 10280, 10036, 10019, 10014, 10038, 10002, 10003,
       10009, 10012, 10016, 10010, 10001, 10011, 10021, 10022, 10020,
       10007, 10128, 11238, 11377, 10024, 10031, 10026, 10027, 10025,
       10033, 10034, 10013, 10065, 10004, 10017])

In [9]:
!unzip taxi_zones.zip

Archive:  taxi_zones.zip
  inflating: taxi_zones.prj          
  inflating: taxi_zones.dbf          
  inflating: taxi_zones.shp          
  inflating: taxi_zones.shp.xml      
  inflating: taxi_zones.shx          
  inflating: taxi_zones.sbn          
  inflating: taxi_zones.sbx          


In [11]:
taxi_zones = gpd.read_file('taxi_zones.shp')

In [12]:
taxi_zones.head()


,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.9183527103 192536.0856972019,..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"(POLYGON ((1033269.243591294 172126.0078125, 1..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.769506663 256767.6975403726,..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.4667968601 203714.0759887695,..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.3104932606 144283.335850656, ..."


In [14]:
taxi_zones.crs

{'proj': 'lcc',
 'lat_1': 40.66666666666666,
 'lat_2': 41.03333333333333,
 'lat_0': 40.16666666666666,
 'lon_0': -74,
 'x_0': 300000,
 'y_0': 0,
 'datum': 'NAD83',
 'units': 'us-ft',
 'no_defs': True}

In [20]:
taxi_zones.to_crs(epsg=4326,inplace=True)

In [16]:
geometry = [Point(xy) for xy in zip(events['venue.location.lon'], events['venue.location.lat'])]
crs = {'init': 'epsg:4326'}
events_geo = gpd.GeoDataFrame(events, crs=crs, geometry=geometry)

In [21]:
events_geo_taxi = gpd.sjoin(events_geo,taxi_zones,op='intersects')

/nfshome/cb4102/.conda/envs/geoplot/lib/python3.7/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')


In [23]:
events_geo_taxi['borough'].unique()

array(['Manhattan', 'Brooklyn', 'Queens'], dtype=object)

In [24]:
events_geo_taxi.drop(['index_right','OBJECTID','Shape_Leng','Shape_Area','zone'],axis=1,inplace=True)

In [26]:
events_geo_taxi.to_csv('events_geo_taxi.csv')

In [27]:
taxi_zones_df = pd.DataFrame(taxi_zones)

In [29]:
taxi_agg_geo = pd.merge(taxi,taxi_zones_df,how='left',left_on=['PULocationID'],right_on='LocationID')

In [32]:
taxi_agg_geo.drop(['OBJECTID','Shape_Leng','Shape_Area','zone','LocationID'],axis=1,inplace=True)

In [34]:
taxi_agg_geo.rename(columns={'geometry':'pickup_geo'},inplace=True)

In [36]:
taxi_agg_geo = pd.merge(taxi_agg_geo,taxi_zones_df,how='left',left_on='DOLocationID',right_on='LocationID')

In [37]:
taxi_agg_geo.drop(['OBJECTID','Shape_Leng','Shape_Area','zone','LocationID'],axis=1,inplace=True)

In [39]:
taxi_agg_geo.rename(columns={'geometry':'dropoff_geo'},inplace=True)

In [41]:
taxi_agg_geo.to_csv('taxi_agg_geo.csv')